In [17]:
import torch

from src.opts.pcg import PCG
from src.models.full_krr import FullKRR
from src.models.inducing_krr import InducingKRR

In [18]:
torch.manual_seed(0)

In [19]:
ntr = 1000
ntst = 100
d = 3

kernel_params = {"type": "rbf", "sigma": 1}
lambd = 1
task = "regression"
device = 'cuda:2'
m = 100

precond_params_nys = {"type": "nystrom", "r": 100, "rho": 1}
precond_params_greedy = {"type": "partial_cholesky", "r": 100, "rho": 1}
precond_params_falkon = {"type": "falkon"}
pcg_iters = 100

In [20]:
xtr = torch.randn(ntr, d).to(device)
xtst = torch.randn(ntst, d).to(device)
btr = torch.randn(ntr).to(device)
btst = torch.randn(ntst).to(device)

### Full KRR

In [21]:
w0 = torch.zeros(ntr).to(device)
model = FullKRR(xtr, btr, xtst, btst, kernel_params, lambd, task, w0, device)
opt = PCG(model, inducing=False, precond_params=precond_params_nys)
opt.run(pcg_iters)

[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
PCG has converged with residual 9.202072419611795e-07 at iteration 8


In [22]:
w0 = torch.zeros(ntr).to(device)
model = FullKRR(xtr, btr, xtst, btst, kernel_params, lambd, task, w0, device)
opt = PCG(model, inducing=False, precond_params=precond_params_greedy)
opt.run(pcg_iters)

PCG has converged with residual 8.849667096910707e-07 at iteration 9


### Inducing KRR

In [23]:
inducing_pts = torch.randperm(xtr.shape[0])[:m]

In [24]:
w0 = torch.zeros(m).to(device)
model = InducingKRR(xtr, btr, xtst, btst, kernel_params, inducing_pts, lambd, task, w0, device)
opt = PCG(model, inducing=True, precond_params=precond_params_falkon)
opt.run(pcg_iters)

PCG has converged with residual 6.76266154187033e-07 at iteration 36
